In [26]:
import pandas as pd
import numpy as np
import math
import folium
import branca
import matplotlib.pyplot as plt
import seaborn as sns
from folium.plugins import MarkerCluster

isFigure = False # choose to plot extra figures

### Now that we have added the climate data into sparse SkiResort dataset we need to combine it with the udpated csv that I've been wokring on adding data into 

In [73]:
df1 = pd.read_excel('SkiResort_update_v03.xlsx',header=0)

# get rid of rows without information on adult fares
df1 = df1.dropna(subset=['adult day pass'])
df1.describe()

,annualSnowfall,lat,long,Avg. Snowfall,runCount,liftCount,longestRun,hourlyLiftCapacity,adult day pass,drop (m),summit (m)
count,101.000000,404.000000,404.000000,56.000000,368.000000,352.00000,174.000000,134.000000,404.000000,235.000000,210.000000
mean,753.640594,44.010896,-94.373505,699.401250,46.508152,7.50000,3509.159511,13591.843284,61.415284,509.742902,2083.655714
std,316.890553,4.489853,19.707811,283.357191,42.817399,5.83144,2457.705788,11863.418215,37.156004,359.348949,2042.484259
min,66.000000,32.452222,-128.953571,86.360000,1.000000,0.00000,2.300000,30.000000,0.000000,30.000000,137.000000
25%,580.000000,40.802649,-113.961806,481.800000,17.750000,4.00000,1820.733120,4825.250000,35.720000,239.000000,807.750000
50%,762.000000,44.028486,-89.818564,711.200000,32.000000,6.00000,3218.687897,10500.000000,52.440000,453.900000,2104.500000
75%,1000.000000,46.626825,-75.445218,899.040000,63.000000,9.00000,4806.750000,16697.250000,79.000000,682.500000,2787.375000
max,1638.000000,55.868826,-54.046468,1270.000000,300.000000,38.00000,16415.308275,61407.000000,199.000000,2612.000000,25485.000000


In [74]:
df2 = pd.read_csv('SkiResort_withTemp.csv')

In [75]:
result = pd.merge(df1,df2,how='left',on=['name','name'])

In [76]:
temp = list(result)
cold_cols = [col for col in result.columns if '_COLD' in col]
result['AVG_GOODDAYS'] = result[cold_cols].sum(axis=1)/result['NYEAR']
# result['COLD_DAYS'] =  

In [77]:
result.to_csv('SkiResort_Combined_test.csv',index=False)

## Exploratory Data Analysis

In [78]:
if isFigure:
    sns.set(style="darkgrid")
    sns.relplot(x="runCount_x", y="drop (m)", size="adult day pass",sizes=(15,200), data=result);


In [79]:
if isFigure:
    g = sns.catplot(x="region", y="adult day pass", kind="swarm", data=result);
    g.set_xticklabels(rotation=90)


## Make a column of the ratio of cold days to total observations


In [80]:
if isFigure:
    sns.relplot(x="lat_x", y="AVG_GOODDAYS", size="adult day pass",sizes=(15,200), hue='region', data=result);

In [81]:
if isFigure:
    g = sns.catplot(x="region", y="AVG_GOODDAYS", kind="swarm", data=result);
    g.set_xticklabels(rotation=90)

## Make some maps

In [82]:
def fancy_html(row):
    
    Region = row['region']
    Name=row['name']
    NRuns=row['runCount_x']
    NLifts=row['liftCount_x']
    Vert = row['drop (m)']
    Good = row['AVG_GOODDAYS']
    Fare = str(int(row['adult day pass']))
    if math.isnan(NRuns):
        NRuns = 'Unknown'
    else:
        NRuns = str(int(NRuns))
        
    if math.isnan(NLifts):
        NLifts = 'Unknown'
    else:
        NLifts = str(int(NLifts))
        
    if math.isnan(Vert):
        Vert = 'Unknown'
    else:
        Vert = str(float(Vert))
    
    if math.isnan(Good):
        Good = 'Unknown'
    else:
        Good = str(int(Good))
        
#     Number_of_Casualties = row['Number_of_Casualties'].iloc[i]                           
#     Date = row['created_x']
#     Time = row['NYEAR']                                    
#     Light_Conditions = df['Light_Conditions'].iloc[i]                               
#     Weather_Conditions = df['Weather_Conditions'].iloc[i]      
#     Road_Surface_Conditions = df['Road_Surface_Conditions'].iloc[i]
    
    left_col_colour = "#2A799C"
    right_col_colour = "#C5DCE7"
    
    html = """<!DOCTYPE html>
<html>

<head>
<h4 style="margin-bottom:0"; width="200px">{}{}</h4>""".format(Name,"<br>" + Region) + """

</head>
    <table style="height: 120px; width: 300px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">No. Runs</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(NRuns) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">No. Lifts</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(NLifts) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Vertical Drop (m)</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(NRuns) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">No. Freezing Days</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Good) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Lift Ticket Price $</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Good) + """
</tr>
</tbody>
</table>
</html>
"""
    return html

In [83]:
m = folium.Map(location=[42,-92], zoom_start = 4.2, tiles = "cartodbpositron")
cluster = MarkerCluster().add_to(m)

for index, row in result.iterrows():
    html = fancy_html(row)
     
    if row['AVG_GOODDAYS'] < 100:
        colr='orange'
#         print('here')
    else:
        colr='green'
    iframe = branca.element.IFrame(html=html,width=400,height=225)
    popup = folium.Popup(iframe,parse_html=True)
#     print(len(row['region']))
    folium.CircleMarker(location=[row['lat_x'], row['long_x']], radius = 9, popup=popup, fill_color=color_change(row['AVG_GOODDAYS']), color="gray", fill_opacity = 0.9).add_to(cluster)
# m.save('SkiResort_map.html')
m